In [27]:
import requests
import json
import pandas as  pd
import time
# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

In [28]:
url = 'http://127.0.0.1:5000/createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [29]:
response.json()

{'signingKey_hex': '7f580e2c2ffc35034a2754701005b0cf6ebcb845c758abc2d3855fb8677460c7',
 'traderId': '1',
 'verifyKey_hex': '51d21b27566b918b61728a757ce23b972c06b757f1dd6b65804dda66fba912b5'}

In [30]:
url = 'http://127.0.0.1:5000/createMarket'

mkId = 1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [31]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [32]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,True,1,1,0,1,1


In [33]:
# Create a trade
url = 'http://127.0.0.1:5000/createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.551, "quantity":1}

In [34]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [35]:
# View json response
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketRootId,price,quantity,traderId
0,"(True, True)",1,1,0.551,1,1


In [36]:
# Create a matched trade
url = 'http://127.0.0.1:5000/createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [37]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 1,
 'price': 0.551,
 'quantity': -1,
 'traderId': '1'}

In [38]:
# Create many matched trades
url = 'http://127.0.0.1:5000/createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 0.7305212020874023 seconds.
matched trade: 1 in 0.6950278282165527 seconds.
matched trade: 2 in 0.6956160068511963 seconds.
matched trade: 3 in 0.7120420932769775 seconds.
matched trade: 4 in 0.7413680553436279 seconds.
matched trade: 5 in 0.7457292079925537 seconds.
matched trade: 6 in 0.7905240058898926 seconds.
matched trade: 7 in 0.7952418327331543 seconds.
matched trade: 8 in 0.8759629726409912 seconds.
matched trade: 9 in 0.7801089286804199 seconds.


In [39]:
url = 'http://127.0.0.1:5000/createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

market created: 1 in 0.12568020820617676 seconds.
market created: 2 in 0.18587088584899902 seconds.
market created: 3 in 0.25840091705322266 seconds.
market created: 4 in 0.4902641773223877 seconds.


In [40]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 1}

In [41]:
url = 'http://127.0.0.1:5000/viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [42]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax
0,1,1,0,0.7
1,2,1,0,0.7
2,3,1,0,0.7
3,4,1,0,0.7


In [43]:
url = 'http://127.0.0.1:5000/viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [44]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.551,1,1
1,1,1,0.551,-1,1
10,1,1,0.551,1,1
11,1,1,0.551,-1,1
12,1,1,0.551,-1,1
13,1,1,0.551,1,1
14,1,1,0.551,-1,1
15,1,1,0.551,1,1
16,1,1,0.551,1,1
17,1,1,0.551,-1,1


In [45]:
# Create many matched trades
url = 'http://127.0.0.1:5000/createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 0.8564910888671875 seconds.
matched trade: 1 in 0.851107120513916 seconds.
matched trade: 2 in 0.8562381267547607 seconds.
matched trade: 3 in 0.8041300773620605 seconds.
matched trade: 4 in 0.8429608345031738 seconds.
matched trade: 5 in 0.9304311275482178 seconds.
matched trade: 6 in 0.8384618759155273 seconds.
matched trade: 7 in 0.8388090133666992 seconds.
matched trade: 8 in 0.8782987594604492 seconds.
matched trade: 9 in 0.8959817886352539 seconds.


In [46]:
url = 'http://127.0.0.1:5000/viewOpenTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [47]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.44,10,1


In [48]:
url = 'http://127.0.0.1:5000/viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [49]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.551,0,1


In [50]:
tId

'1'